# ML4FG Interim Report HQNN Code
### By: Austin Stiefelmaier 11/11/23

## Imports and Settings

In [ ]:
# Inspired by quantum ML code from:
# https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html
# And classical ML code from:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [42]:
# General imports
import os
import copy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import TensorDataset

# Pennylane imports
import pennylane as qml
from pennylane import numpy as np

In [2]:
# Fix random num generation for reproducibility 
torch.manual_seed(7)
np.random.seed(7)
# os.environ["OMP_NUM_THREADS"] = "1"
# If GPU available, set to run on it
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


## Load Data

In [3]:
# Data download
df_x = pd.read_csv('/home/as6734/ml4fg_class_project/TCGA-PANCAN-HiSeq-801x20531/data.csv')
df_y = pd.read_csv('/home/as6734/ml4fg_class_project/TCGA-PANCAN-HiSeq-801x20531/labels.csv')

In [4]:
df_x.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [5]:
df_y.head()

,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA


In [6]:
# Clean data

# Drop first column that only notes sample number (which can be reconstructed from index if need be
df_x.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df_y.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# Remove columns with all 0.0 values
df_x = df_x.loc[:, (df_x != 0).any(axis=0)]

In [7]:
# Pre-process data

# Normalize values by mean
df_x=(df_x-df_x.mean())/df_x.std()
# Encode classes
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(df_y.values)
y = enc.transform(df_y.values)

In [8]:
df_x.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_6,gene_7,gene_8,gene_9,gene_10,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,-0.194678,-0.827513,0.159701,-1.947061,1.220812,-0.207838,0.180797,-0.125297,-0.065592,-0.082063,...,-1.299388,-0.921179,-0.877290,0.995625,-1.165344,0.389198,-0.869023,-1.187196,-0.116410,-0.261738
1,-0.194678,-2.013759,-1.414158,1.352264,-0.376283,-0.531890,-0.982474,-0.125297,-0.065592,-0.586397,...,-1.745985,-2.390720,-0.831373,0.591280,-2.548006,1.390759,0.623162,-0.342063,-1.655854,-0.261738
2,-0.194678,0.417087,1.156013,0.249651,0.112761,-0.391053,-0.092937,-0.125297,-0.065592,-0.586397,...,-1.033442,-1.059008,2.247399,0.232456,0.317682,-4.023107,-0.631986,0.886307,-1.854106,-0.261738
3,-0.194678,0.543549,1.325354,-0.098991,0.755269,0.395101,-0.127752,-0.125297,-0.065592,-0.586397,...,0.241148,0.044877,0.224815,1.718651,-0.263682,-0.521421,-0.793113,0.166070,-1.078268,-0.261738
4,-0.194678,-0.295770,-0.256947,-0.286234,-0.148750,-0.756645,-0.272995,-0.125297,-0.065592,-0.586397,...,0.133251,0.208122,0.837216,0.979312,0.196522,0.268824,-1.614832,-0.229734,-0.201463,-0.261738


In [9]:
y[0:5]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [10]:
# Split data, final percentages are approximately 64% train, 16% validation, 20% test
x_train, x_test, y_train, y_test = train_test_split(df_x.values, y, test_size=0.2, train_size=0.8, random_state=7, stratify=y)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, train_size=0.8, random_state=7, stratify=y_train)
dataset_sizes = {'train': len(x_train), 'validation': len(x_valid), 'test': len(x_test)}
print(dataset_sizes)

{'train': 512, 'validation': 128, 'test': 161}


In [27]:
# Data split analysis

def sort_and_prepprint_dict(input_dict, ints=True):
    outstring = ''
    # Sort the dictionary by keys in alphabetical order
    sorted_dict = {k: input_dict[k] for k in sorted(input_dict)}
    # Print key-value pairs with 3 decimal place precision for float values
    for key, value in sorted_dict.items():
        if ints:
            outstring += f"{key}: {value}, "
        else:
            outstring += f"{key}: {value:.3f}, "
    return outstring[:-2]

def get_data_counts(data, datatype, total):
    cats = {}
    cats_pcts = {}
    for i in enc.inverse_transform(data):
        if i[0] not in cats:
            cats[i[0]] = 1
            cats_pcts[i[0]] = 1
        else:
            cats[i[0]] += 1
            cats_pcts[i[0]] += 1
    print(f'{datatype} Data Categories (total={total}):')
    print(sort_and_prepprint_dict(cats))
    for key in cats_pcts:
        cats_pcts[key] /= total
    print(sort_and_prepprint_dict(cats_pcts, ints=False))

get_data_counts(y_train, 'Train', dataset_sizes['train'])
print()
get_data_counts(y_valid, 'Validation', dataset_sizes['validation'])
print()
get_data_counts(y_test, 'Test', dataset_sizes['test'])
print()
get_data_counts(y, 'All', len(y))

Train Data Categories (total=512):
BRCA: 192, COAD: 50, KIRC: 93, LUAD: 90, PRAD: 87
BRCA: 0.375, COAD: 0.098, KIRC: 0.182, LUAD: 0.176, PRAD: 0.170

Validation Data Categories (total=128):
BRCA: 48, COAD: 12, KIRC: 23, LUAD: 23, PRAD: 22
BRCA: 0.375, COAD: 0.094, KIRC: 0.180, LUAD: 0.180, PRAD: 0.172

Test Data Categories (total=161):
BRCA: 60, COAD: 16, KIRC: 30, LUAD: 28, PRAD: 27
BRCA: 0.373, COAD: 0.099, KIRC: 0.186, LUAD: 0.174, PRAD: 0.168

All Data Categories (total=801):
BRCA: 300, COAD: 78, KIRC: 146, LUAD: 141, PRAD: 136
BRCA: 0.375, COAD: 0.097, KIRC: 0.182, LUAD: 0.176, PRAD: 0.170


In [28]:
# Convert to DataLoaders
x_train_to_tensor = torch.from_numpy(x_train).to(torch.float32).to(device)
y_train_to_tensor = torch.from_numpy(y_train).to(torch.float32).to(device)
x_valid_to_tensor = torch.from_numpy(x_valid).to(torch.float32).to(device)
y_valid_to_tensor = torch.from_numpy(y_valid).to(torch.float32).to(device)
x_test_to_tensor = torch.from_numpy(x_test).to(torch.float32).to(device)
y_test_to_tensor = torch.from_numpy(y_test).to(torch.float32).to(device)

# Second step: Creating TensorDataset for Dataloader
train_set = TensorDataset(x_train_to_tensor, y_train_to_tensor)
valid_set = TensorDataset(x_valid_to_tensor, y_valid_to_tensor)
test_set = TensorDataset(x_test_to_tensor, y_test_to_tensor)

# Create DataLoader
dataloaders = {
    'train': torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True),
    'validation': torch.utils.data.DataLoader(valid_set, batch_size=8),
    'test': torch.utils.data.DataLoader(test_set, shuffle=True, batch_size=1)
}

## Quantum Circuit Architecture

In [29]:
# Quantum circuit params
qubit_count = 5
circ_repeats = 6  # How many times to repeat RY and CNOT gates
q_delta = 0.01  # Param for quantum circuit weight initialization

# Run quantum circuit on Pennylane default simulator
dev = qml.device('default.qubit', wires=qubit_count)

In [30]:
# Helper functions to construct quantum circuit
def ry_gates(w):
    # Apply rotation gate RY w/ given weights
    for i, weight in enumerate(w):
        qml.RY(weight, wires=i)

# Adds alternating CNOT layer to entangle qubits
def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):  # Evens
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Odds
        qml.CNOT(wires=[i, i + 1])

In [31]:
# Function to construct quantum circuit to plug into PyTorch
@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, q_weights_flat):
    # Reshape weights
    q_weights = q_weights_flat.reshape(circ_repeats, qubit_count)
    # Initialize w/ H gates so orthogonal to computational basis
    # This helps start w/out bias towards 0 or 1 states
    for i in range(qubit_count):
        qml.Hadamard(wires=i)
    # Take given inputs and apply to quantum circuit as first weights
    ry_gates(inputs)
    # Repeat CNOT and RY gates to add more weights to train and
    # CNOT "convolutions" (really entangles)
    for k in range(circ_repeats):
        entangling_layer(qubit_count)
        ry_gates(q_weights[k])
    # Use Pennylane sim to get expected value after applying Z gate
    # which returns to standard computational basis, this is layer output
    expected_vals = [qml.expval(qml.PauliZ(entry)) for entry in range(qubit_count)]
    return tuple(expected_vals)

In [32]:
# Example visualization codes with nonsense weights
# Helps to see what the quantum circuit looks like
q_params_print = nn.Parameter(q_delta * torch.randn(circ_repeats * qubit_count))
input_test = torch.randn(1,512)
pre_net_test = nn.Linear(512, qubit_count)
pre_out_test = pre_net_test(input_test)
q_in_test = torch.tanh(pre_out_test) * np.pi / 2.0
for elem in q_in_test:
    print(qml.draw(quantum_circuit)(elem, q_params_print))

0: ──H──RY(-0.15)─╭●──RY(-0.01)────────────╭●──RY(0.00)────────────╭●──RY(-0.01)────────────╭●
1: ──H──RY(0.69)──╰X─╭●──────────RY(0.00)──╰X─╭●─────────RY(-0.01)─╰X─╭●──────────RY(-0.00)─╰X
2: ──H──RY(0.21)──╭●─╰X──────────RY(0.01)──╭●─╰X─────────RY(-0.01)─╭●─╰X──────────RY(0.01)──╭●
3: ──H──RY(-0.84)─╰X─╭●──────────RY(-0.01)─╰X─╭●─────────RY(0.01)──╰X─╭●──────────RY(0.01)──╰X
4: ──H──RY(-0.98)────╰X──────────RY(-0.00)────╰X─────────RY(-0.01)────╰X──────────RY(-0.01)───

───RY(0.01)────────────╭●──RY(0.01)────────────╭●──RY(0.01)────────────┤  <Z>
──╭●─────────RY(-0.00)─╰X─╭●─────────RY(-0.01)─╰X─╭●─────────RY(0.02)──┤  <Z>
──╰X─────────RY(-0.00)─╭●─╰X─────────RY(0.01)──╭●─╰X─────────RY(-0.01)─┤  <Z>
──╭●─────────RY(0.00)──╰X─╭●─────────RY(0.01)──╰X─╭●─────────RY(-0.00)─┤  <Z>
──╰X─────────RY(0.02)─────╰X─────────RY(-0.01)────╰X─────────RY(-0.00)─┤  <Z>


## HQNN Architecture

In [33]:
# HQNN Model Class
class HybridQuantumNet(nn.Module):
    # Initialize layers
    def __init__(self, num_feature):
        super().__init__()
        # Layers before quantum circuit
        self.layer_1 = nn.Linear(num_feature, 512)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.layer_2 = nn.Linear(512, 128)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.layer_3 = nn.Linear(128, 64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.layer_4 = nn.Linear(64, qubit_count)
        # Quantum Circuit params
        self.q_params = nn.Parameter(q_delta * torch.randn(circ_repeats * qubit_count))
        # Layer(s) after quantum circuit
        # self.post_quant = torch.nn.Softmax(dim=qubit_count)
        # self.post_quant = nn.Linear(qubit_count, 5)
        # Misc
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)

    # Forward pass procedure
    def forward(self, x):
        # First layer
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)

        # Second layer
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)

        # Third layer
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Map classical 64 output to qubit_count output
        pre_out = self.layer_4(x)
        
        # Convert to radians for use as RY rotation gate params
        q_in = torch.tanh(pre_out) * np.pi / 2.0
        # Apply quantum circuit set to run on GPU
        q_out = torch.Tensor(0, qubit_count)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_circuit(elem, self.q_params)
            q_out_elem = torch.stack(list(q_out_elem), dim=0)
            q_out_elem = q_out_elem.float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))
        # Map quantum circuit output using softmax to classes
        # return self.post_quant(q_out)
        return q_out

In [34]:
# Instantiate model
model_hybrid = HybridQuantumNet(num_feature=len(df_x.columns))
# Make sure set to use GPU
model_hybrid = model_hybrid.to(device)
# Show model architecture summary
print(model_hybrid)

HybridQuantumNet(
  (layer_1): Linear(in_features=20264, out_features=512, bias=True)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_4): Linear(in_features=64, out_features=5, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)


## Train HQNN

In [35]:
step = 0.0004  # Initial learning rate
batch_size = 8
num_epochs = 50
gamma_lr_scheduler = 0.1  # Learning rate decay param

In [36]:
# Set up loss, optimizer, and learning rate decay manager
loss_func = nn.CrossEntropyLoss()
optimizer_hybrid = optim.Adam(model_hybrid.parameters(), lr=step)
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
)

In [39]:
# Function to train model(s)
def train(model, loss_func, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000.0
    best_loss_train = 100000.0
    print('Training started:')
    for epoch in tqdm(range(num_epochs)):
        for phase in ['train', 'validation']:
            if phase == 'train':
                # Set model to train mode
                model.train()
            else:
                # Set model to eval mode
                model.eval()
            current_loss = 0.0
            n_batches = dataset_sizes[phase] // batch_size
            iter = 0
            for X, Y in dataloaders[phase]:
                batch_len = len(X)
                X = X.to(device)
                optimizer.zero_grad()  # Reset gradients
                # If in train mode, get loss, step optimizer
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(X)
                    _, preds = torch.max(outputs, 1)
                    loss = loss_func(outputs, Y)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                # Print iteration results
                current_loss += loss.item() * batch_len
                print('Phase: {} Epoch: {}/{} Iter: {}/{}'.format(phase, epoch+1, num_epochs, iter+1, n_batches+1),
                    end="\r",
                    flush=True)
                iter += 1

            # Get epoch stats and print
            epoch_loss = current_loss / dataset_sizes[phase]
            print('Phase: {} Epoch: {}/{} Loss: {:.4f}        '.format(
                    'train' if phase == 'train' else 'validation  ',
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                )
            )
            # Update best vars and make copy of best weights
            # if phase == "validation":
            #     best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "validation" and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss
            # Decay learning rate
            if phase == "train":
                scheduler.step()
    # Print final results
    model.load_state_dict(best_model_wts)
    print('Best test loss: {:.4f}'.format(best_loss))
    return model

In [40]:
model_hybrid = train(model_hybrid, loss_func, optimizer_hybrid, exp_lr_scheduler, num_epochs=num_epochs)

Training started:


  0%|          | 0/50 [00:00<?, ?it/s]

Phase: train Epoch: 1/50 Loss: 0.5953        
Phase: validation   Epoch: 1/50 Loss: 0.5217        
Phase: train Epoch: 2/50 Loss: 0.5982        
Phase: validation   Epoch: 2/50 Loss: 0.5225        
Phase: train Epoch: 3/50 Loss: 0.6104        
Phase: validation   Epoch: 3/50 Loss: 0.5232        
Phase: train Epoch: 4/50 Loss: 0.5845        
Phase: validation   Epoch: 4/50 Loss: 0.5239        
Phase: train Epoch: 5/50 Loss: 0.5742        
Phase: validation   Epoch: 5/50 Loss: 0.5221        
Phase: train Epoch: 6/50 Loss: 0.5702        
Phase: validation   Epoch: 6/50 Loss: 0.5224        
Phase: train Epoch: 7/50 Loss: 0.5702        
Phase: validation   Epoch: 7/50 Loss: 0.5251        
Phase: train Epoch: 8/50 Loss: 0.5833        
Phase: validation   Epoch: 8/50 Loss: 0.5206        
Phase: train Epoch: 9/50 Loss: 0.5769        
Phase: validation   Epoch: 9/50 Loss: 0.5223        
Phase: train Epoch: 10/50 Loss: 0.5625        
Phase: validation   Epoch: 10/50 Loss: 0.5228        


KeyboardInterrupt: 

In [41]:
# Save model weights locally in case of GCP crash
torch.save(model_hybrid.state_dict(), './weights/hybrid_50epochs.pt')

## Test HQNN

In [ ]:
# Load in saved weights, put in model, and than set to eval mode
best_weights = torch.load('./weights/hybrid_50epochs.pt')
model_hybrid.load_state_dict(best_weights)
model_hybrid.eval()

In [52]:
# Helper function to test model
def test(model, device, test_loader):
    preds = []
    targets = []
    with torch.no_grad():
        model.eval()
        for data, target in tqdm(test_loader):
            # Send the data and target to device
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds.append(enc.inverse_transform(output)[0])
            targets.append(enc.inverse_transform(target)[0])
    return preds, targets

preds, targets = test(model_hybrid, 'cpu', dataloaders['test'])

  0%|          | 0/161 [00:00<?, ?it/s]

In [54]:
print(classification_report(y_true=targets, y_pred=preds))

              precision    recall  f1-score   support

        BRCA       0.78      1.00      0.88        60
        COAD       1.00      1.00      1.00        16
        KIRC       1.00      0.43      0.60        30
        LUAD       1.00      1.00      1.00        28
        PRAD       1.00      1.00      1.00        27

    accuracy                           0.89       161
   macro avg       0.96      0.89      0.90       161
weighted avg       0.92      0.89      0.88       161

